In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import cv2

In [2]:
df = pd.read_csv("C:\Subjects\Subjects\Object Extraction Project\dataframe_reduced.csv")
#print(df)

def transform_IMG(img_path):
    image = cv2.imread(img_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return image

class Dataset_dl(Dataset):
    def __init__(self, paths, x1, y1, x2, y2, x3, y3, x4, y4, y):
        self.paths = paths.values
        self.x1 = x1.values
        self.y1 = y1.values
        self.x2 = x2.values
        self.y2 = y2.values
        self.x3 = x3.values
        self.y3 = y3.values
        self.x4 = x4.values
        self.y4 = y4.values
        #self.y_bb = [X['x1'], X['y1'], X['x2'], X['y2'], X['x3'], X['y3'], X['x4'], X['y4']]
        #self.y_bb = np.array(list(zip(X['x1'], X['y1'], X['x2'], X['y2'], X['x3'], X['y3'], X['x4'], X['y4'])))
        #X = transform_IMG(self.path)
        self.y = y.values
        #self.num_samples = X.shape[0]
        #print(len(self.path), len(self.y_bb), len(self.y))
        #print(self.num_samples)
        
    def __len__(self):
        return len(self.paths)
    
    def __getitem__(self, index):
        #print(index)
        path = self.paths[index]
        y_class = self.y[index]
        x1_bb = self.x1[index]
        y1_bb = self.y1[index]
        x2_bb = self.x2[index]
        y2_bb = self.y2[index]
        x3_bb = self.x3[index]
        y3_bb = self.y3[index]
        x4_bb = self.x4[index]
        y4_bb = self.y4[index]
        
        #y_bb = self.y_bb[index]
        x = transform_IMG(path)
        x = np.expand_dims(x, axis=-1)
        x = np.transpose(x, (2, 0, 1))
        #print(x.shape)
        
        #x = x.transpose(0,1,2)

        return x, y_class, x1_bb, y1_bb, x2_bb, y2_bb, x3_bb, y3_bb, x4_bb, y4_bb

In [3]:
X = df[['image_path','x1', 'y1','x2', 'y2','x3', 'y3','x4', 'y4']]
y = df['outcome']

#X = torch.from_numpy(np.array(X['x1'].values.tolist(), np.float))
#Y = torch.from_numpy(np.array(Y['outcome'].values.tolist(), np.float))
print(X)
print(y)

                                             image_path          x1  \
0     C:\Subjects\Subjects\Object Extraction Project...  889.951975   
1     C:\Subjects\Subjects\Object Extraction Project...  183.225407   
2     C:\Subjects\Subjects\Object Extraction Project...  918.506584   
3     C:\Subjects\Subjects\Object Extraction Project...  912.161115   
4     C:\Subjects\Subjects\Object Extraction Project...  908.195197   
...                                                 ...         ...   
9800  C:\Subjects\Subjects\Object Extraction Project...  251.591274   
9801  C:\Subjects\Subjects\Object Extraction Project...  209.267509   
9802  C:\Subjects\Subjects\Object Extraction Project...  877.042480   
9803  C:\Subjects\Subjects\Object Extraction Project...  898.204363   
9804  C:\Subjects\Subjects\Object Extraction Project...  833.543054   

               y1          x2           y2          x3           y3  \
0      900.606325  893.124709   897.256270  905.815647   901.164667   
1    

In [4]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)
df_train = Dataset_dl(X_train['image_path'], X_train['x1'], X_train['y1'], X_train['x2'], X_train['y2'], X_train['x3'], X_train['y3'], X_train['x4'], X_train['y4'], y_train)
df_val = Dataset_dl(X_val['image_path'], X_val['x1'], X_val['y1'], X_val['x2'], X_val['y2'], X_val['x3'], X_val['y3'], X_val['x4'], X_val['y4'], y_val)

In [5]:
batch_size = 64
train_dl = DataLoader(df_train, batch_size=batch_size, shuffle=True)
valid_dl = DataLoader(df_val, batch_size=batch_size)

In [71]:
'''class BB_model(nn.Module):
    def __init__(self):
        super(BB_model, self).__init__()
        resnet = models.resnet34(pretrained=True)
        # Modify the first convolutional layer to accept a single channel input
        # Change the first Conv2d layer's in_channels from 3 to 1
        resnet.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        
        layers = list(resnet.children())[:8]
        self.features1 = nn.Sequential(*layers[:6])
        self.features2 = nn.Sequential(*layers[6:])
        self.classifier = nn.Sequential(nn.BatchNorm1d(512), nn.Linear(512, 15))
        self.bb = nn.Sequential(nn.BatchNorm1d(512), nn.Linear(512, 8))
        
    def forward(self, x):
        x = self.features1(x)
        x = self.features2(x)
        x = F.relu(x)
        x = nn.AdaptiveAvgPool2d((1, 1))(x)
        x = x.view(x.shape[0], -1)
        return self.classifier(x), self.bb(x)'''
    
class BB_model(nn.Module):
    def __init__(self):
        super(BB_model, self).__init__()
        # Define a very basic neural network with minimal layers
        self.features = nn.Sequential(
            nn.Linear(1 * 1024 * 1024, 128),
            nn.ReLU()
        )
        
        # Output layers similar to the BB_model
        self.classifier = nn.Sequential(nn.Linear(128, 16),
                                        nn.Softmax(dim=1))
        self.bb = nn.Linear(128, 8)

    def forward(self, x):
        x = x.view(x.size(0), -1)  # Flatten the tensor
        x = self.features(x)
        class_output = self.classifier(x)
        bb_output = self.bb(x)
        return class_output, bb_output


In [72]:
def update_optimizer(optimizer, lr):
    for i, param_group in enumerate(optimizer.param_groups):
        param_group["lr"] = lr

def train_epocs(model, optimizer, train_dl, val_dl, epochs=1,C=1000):
    idx = 0
    for i in range(epochs):
        model.train()
        total = 0
        sum_loss = 0
        for x_batch, y_class, x1, y1, x2, y2, x3, y3, x4, y4 in train_dl:
            batch = y_class.shape[0]
            #x = x_batch.float()
            #y_class = y_class
            
            x1 = x1.float()
            y1 = y1.float()
            x2 = x2.float()
            y2 = y2.float()
            x3 = x3.float()
            y3 = y3.float()
            x4 = x4.float()
            y4 = y4.float()
            y_bb = torch.cat([x1.unsqueeze(1), y1.unsqueeze(1),
                              x2.unsqueeze(1), y2.unsqueeze(1),
                              x3.unsqueeze(1), y3.unsqueeze(1),
                              x4.unsqueeze(1), y4.unsqueeze(1)], dim=1)
            
            out_class, out_bb = model(x_batch.float())
            loss_class = F.cross_entropy(out_class, y_class, reduction="sum")
            loss_bb = F.l1_loss(out_bb, y_bb, reduction="none").sum(1)
            loss_bb = loss_bb.sum()
            loss = loss_class + loss_bb/C
            print(loss.item())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            idx += 1
            total += batch
            sum_loss += loss.item()
        train_loss = sum_loss/total
        val_loss, val_acc = val_metrics(model, val_dl, C)
        print("train_loss %.3f val_loss %.3f val_acc %.3f" % (train_loss, val_loss, val_acc))
    return sum_loss/total  

def val_metrics(model, valid_dl, C=1000):
    model.eval()
    total = 0
    sum_loss = 0
    correct = 0 
    for x_batch, y_class, x1, y1, x2, y2, x3, y3, x4, y4 in valid_dl:
        batch = y_class.shape[0]
        #x = x.float()
        y_class = y_class
        x1 = x1.float()
        y1 = y1.float()
        x2 = x2.float()
        y2 = y2.float()
        x3 = x3.float()
        y3 = y3.float()
        x4 = x4.float()
        y4 = y4.float()
        y_bb = torch.cat([x1.unsqueeze(1), y1.unsqueeze(1),
                          x2.unsqueeze(1), y2.unsqueeze(1),
                          x3.unsqueeze(1), y3.unsqueeze(1),
                          x4.unsqueeze(1), y4.unsqueeze(1)], dim=1)
        out_class, out_bb = model(x_batch.float())
        loss_class = F.cross_entropy(out_class, y_class, reduction="sum")
        loss_bb = F.l1_loss(out_bb, y_bb, reduction="none").sum(1)
        loss_bb = loss_bb.sum()
        loss = loss_class + loss_bb/C
        _, pred = torch.max(out_class, 1)
        correct += pred.eq(y_class).sum().item()
        sum_loss += loss.item()
        total += batch
    return sum_loss/total, correct/total

In [73]:
model = BB_model()
parameters = filter(lambda p: p.requires_grad, model.parameters())
optimizer = torch.optim.Adam(parameters, lr=0.006)

print(train_dl)
train_epocs(model, optimizer, train_dl, valid_dl, epochs=1)
'''
update_optimizer(optimizer, 0.001)
train_epocs(model, optimizer, train_dl, valid_dl, epochs=10)'''

432.15557861328125
340999.40625
161541.171875
39738.23046875
48450.9609375
35669.15625
24663.09765625
17631.5546875
6668.68603515625
1634.3380126953125
426.6846923828125
455.9427490234375
440.4803466796875
419.3961181640625
418.9139404296875
435.81707763671875
439.1587829589844
438.5629577636719
441.3668518066406
441.005615234375
432.533935546875
447.33807373046875
440.5169677734375
430.59869384765625
457.1484069824219
418.42572021484375
437.07159423828125
407.59326171875
428.60137939453125
432.1031188964844
433.93597412109375
430.5701904296875
442.447265625
437.8330078125
437.298095703125
433.915771484375
451.1473388671875
417.54144287109375
434.162353515625
469.72412109375
415.5771484375
446.7440185546875
408.8853759765625
435.80010986328125
448.3980712890625
438.35455322265625
433.79046630859375
420.64263916015625
436.31109619140625
439.46197509765625
417.69403076171875
444.5187683105469
428.122802734375
409.8580322265625
419.7960205078125
441.1412353515625
409.65289306640625
424.48

'\nupdate_optimizer(optimizer, 0.001)\ntrain_epocs(model, optimizer, train_dl, valid_dl, epochs=10)'

In [74]:
X_val['image_path'].iloc[0]

'C:\\Subjects\\Subjects\\Object Extraction Project\\images_resized\\P0052_3.png'

In [75]:
im = 'C:\\Subjects\\Subjects\\Object Extraction Project\\images_resized\\P0052_3.png'
X_val['image_path']

6572    C:\Subjects\Subjects\Object Extraction Project...
7668    C:\Subjects\Subjects\Object Extraction Project...
3953    C:\Subjects\Subjects\Object Extraction Project...
8385    C:\Subjects\Subjects\Object Extraction Project...
741     C:\Subjects\Subjects\Object Extraction Project...
                              ...                        
8595    C:\Subjects\Subjects\Object Extraction Project...
9474    C:\Subjects\Subjects\Object Extraction Project...
2800    C:\Subjects\Subjects\Object Extraction Project...
2744    C:\Subjects\Subjects\Object Extraction Project...
9092    C:\Subjects\Subjects\Object Extraction Project...
Name: image_path, Length: 1961, dtype: object

In [76]:
#X = df[['image_path','x1', 'y1','x2', 'y2','x3', 'y3','x4', 'y4']]

test_ds = Dataset_dl(pd.DataFrame([{'image_path':im}])['image_path'],pd.DataFrame([{'x1':0}])['x1'],\
                      pd.DataFrame([{'y1':0}])['y1'], pd.DataFrame([{'x2':0}])['x2'] , pd.DataFrame([{'y2':0}])['y2'],\
                      pd.DataFrame([{'x3':0}])['x3'],pd.DataFrame([{'y3':0}])['y3'], pd.DataFrame([{'x4':0}])['x4'], \
                      pd.DataFrame([{'y4':0}])['y4'] , pd.DataFrame([{'y':[0]}])['y'])
x, y_class, x1,y1, x2, y2, x3, y3, x4, y4 = test_ds[0]

In [77]:
xx = torch.FloatTensor(x[None,])
xx.shape

torch.Size([1, 1, 1024, 1024])

In [78]:
out_class, out_bb = model(xx)
out_class, out_bb

(tensor([[0.2298, 0.0475, 0.0412, 0.1240, 0.0431, 0.0420, 0.0432, 0.0649, 0.0474,
          0.0473, 0.0417, 0.0473, 0.0462, 0.0448, 0.0445, 0.0452]],
        grad_fn=<SoftmaxBackward0>),
 tensor([[0.6712, 0.5732, 0.6482, 0.6979, 0.7147, 0.7270, 0.6842, 0.5940]],
        grad_fn=<AddmmBackward0>))

In [79]:
torch.max(out_class, 1)

torch.return_types.max(
values=tensor([0.2298], grad_fn=<MaxBackward0>),
indices=tensor([0]))

In [80]:
out_bb

tensor([[0.6712, 0.5732, 0.6482, 0.6979, 0.7147, 0.7270, 0.6842, 0.5940]],
       grad_fn=<AddmmBackward0>)

In [81]:
y_preds = []
for x_batch, y_class, x1, y1, x2, y2, x3, y3, x4, y4 in valid_dl:
        batch = y_class.shape[0]
        #x = x.float()
        y_class = y_class
        x1 = x1.float()
        y1 = y1.float()
        x2 = x2.float()
        y2 = y2.float()
        x3 = x3.float()
        y3 = y3.float()
        x4 = x4.float()
        y4 = y4.float()
        y_bb = torch.cat([x1.unsqueeze(1), y1.unsqueeze(1),
                          x2.unsqueeze(1), y2.unsqueeze(1),
                          x3.unsqueeze(1), y3.unsqueeze(1),
                          x4.unsqueeze(1), y4.unsqueeze(1)], dim=1)
        out_class, out_bb = model(x_batch.float())
        y_preds.append(out_class)
        print("out_class", out_class)
        print("pred_class: ",torch.max(out_class, 1))
        print("y_class: ",y_class)
#print(len(y_preds))

out_class tensor([[0.2298, 0.0475, 0.0412,  ..., 0.0448, 0.0445, 0.0452],
        [0.2298, 0.0475, 0.0412,  ..., 0.0448, 0.0445, 0.0452],
        [0.2298, 0.0475, 0.0412,  ..., 0.0448, 0.0445, 0.0452],
        ...,
        [0.2298, 0.0475, 0.0412,  ..., 0.0448, 0.0445, 0.0452],
        [0.2298, 0.0475, 0.0412,  ..., 0.0448, 0.0445, 0.0452],
        [0.2298, 0.0475, 0.0412,  ..., 0.0448, 0.0445, 0.0452]],
       grad_fn=<SoftmaxBackward0>)
pred_class:  torch.return_types.max(
values=tensor([0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298,
        0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298,
        0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298,
        0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298,
        0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298,
        0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298,
        0.2298, 0.2298, 0.2298, 0

out_class tensor([[0.2298, 0.0475, 0.0412,  ..., 0.0448, 0.0445, 0.0452],
        [0.2298, 0.0475, 0.0412,  ..., 0.0448, 0.0445, 0.0452],
        [0.2298, 0.0475, 0.0412,  ..., 0.0448, 0.0445, 0.0452],
        ...,
        [0.2298, 0.0475, 0.0412,  ..., 0.0448, 0.0445, 0.0452],
        [0.2298, 0.0475, 0.0412,  ..., 0.0448, 0.0445, 0.0452],
        [0.2298, 0.0475, 0.0412,  ..., 0.0448, 0.0445, 0.0452]],
       grad_fn=<SoftmaxBackward0>)
pred_class:  torch.return_types.max(
values=tensor([0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298,
        0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298,
        0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298,
        0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298,
        0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298,
        0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298,
        0.2298, 0.2298, 0.2298, 0

out_class tensor([[0.2298, 0.0475, 0.0412,  ..., 0.0448, 0.0445, 0.0452],
        [0.2298, 0.0475, 0.0412,  ..., 0.0448, 0.0445, 0.0452],
        [0.2298, 0.0475, 0.0412,  ..., 0.0448, 0.0445, 0.0452],
        ...,
        [0.2298, 0.0475, 0.0412,  ..., 0.0448, 0.0445, 0.0452],
        [0.2298, 0.0475, 0.0412,  ..., 0.0448, 0.0445, 0.0452],
        [0.2298, 0.0475, 0.0412,  ..., 0.0448, 0.0445, 0.0452]],
       grad_fn=<SoftmaxBackward0>)
pred_class:  torch.return_types.max(
values=tensor([0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298,
        0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298,
        0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298,
        0.2298, 0.2298, 0.2298, 0.2298, 1.0000, 0.2298, 0.2298, 0.2298, 0.2298,
        0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298,
        0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298,
        0.2298, 0.2298, 0.2298, 0

out_class tensor([[0.2298, 0.0475, 0.0412,  ..., 0.0448, 0.0445, 0.0452],
        [0.2298, 0.0475, 0.0412,  ..., 0.0448, 0.0445, 0.0452],
        [0.2298, 0.0475, 0.0412,  ..., 0.0448, 0.0445, 0.0452],
        ...,
        [0.2298, 0.0475, 0.0412,  ..., 0.0448, 0.0445, 0.0452],
        [0.2298, 0.0475, 0.0412,  ..., 0.0448, 0.0445, 0.0452],
        [0.2298, 0.0475, 0.0412,  ..., 0.0448, 0.0445, 0.0452]],
       grad_fn=<SoftmaxBackward0>)
pred_class:  torch.return_types.max(
values=tensor([0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298,
        0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298,
        0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298,
        0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298,
        0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298,
        0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298,
        0.2298, 0.2298, 0.2298, 0

out_class tensor([[0.2298, 0.0475, 0.0412,  ..., 0.0448, 0.0445, 0.0452],
        [0.2298, 0.0475, 0.0412,  ..., 0.0448, 0.0445, 0.0452],
        [0.2298, 0.0475, 0.0412,  ..., 0.0448, 0.0445, 0.0452],
        ...,
        [0.2298, 0.0475, 0.0412,  ..., 0.0448, 0.0445, 0.0452],
        [0.2298, 0.0475, 0.0412,  ..., 0.0448, 0.0445, 0.0452],
        [0.2298, 0.0475, 0.0412,  ..., 0.0448, 0.0445, 0.0452]],
       grad_fn=<SoftmaxBackward0>)
pred_class:  torch.return_types.max(
values=tensor([0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298,
        0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298,
        0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298,
        0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298,
        0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298,
        0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298, 0.2298,
        0.2298, 0.2298, 0.2298, 0

out_class tensor([[0.2298, 0.0475, 0.0412, 0.1240, 0.0431, 0.0420, 0.0432, 0.0649, 0.0474,
         0.0473, 0.0417, 0.0473, 0.0462, 0.0448, 0.0445, 0.0452],
        [0.2298, 0.0475, 0.0412, 0.1240, 0.0431, 0.0420, 0.0432, 0.0649, 0.0474,
         0.0473, 0.0417, 0.0473, 0.0462, 0.0448, 0.0445, 0.0452],
        [0.2298, 0.0475, 0.0412, 0.1240, 0.0431, 0.0420, 0.0432, 0.0649, 0.0474,
         0.0473, 0.0417, 0.0473, 0.0462, 0.0448, 0.0445, 0.0452],
        [0.2298, 0.0475, 0.0412, 0.1240, 0.0431, 0.0420, 0.0432, 0.0649, 0.0474,
         0.0473, 0.0417, 0.0473, 0.0462, 0.0448, 0.0445, 0.0452],
        [0.2298, 0.0475, 0.0412, 0.1240, 0.0431, 0.0420, 0.0432, 0.0649, 0.0474,
         0.0473, 0.0417, 0.0473, 0.0462, 0.0448, 0.0445, 0.0452],
        [0.2298, 0.0475, 0.0412, 0.1240, 0.0431, 0.0420, 0.0432, 0.0649, 0.0474,
         0.0473, 0.0417, 0.0473, 0.0462, 0.0448, 0.0445, 0.0452],
        [0.2298, 0.0475, 0.0412, 0.1240, 0.0431, 0.0420, 0.0432, 0.0649, 0.0474,
         0.0473, 0.0417, 0.